# Análisis exploratorio de datos - Educación

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Load data
path = '/work/covid-ec/data/processed/'

closures_education = pd.read_csv(path + 'closures_education.csv')
education_ec = pd.read_csv(path + 'education_ec.csv')

In [5]:
# Transform to datetime
closures_education['Date'] = pd.to_datetime(closures_education['Date'])

## UNESCO

Puedes categorizar por:
- Nivel educativo
- Sostenimiento (particular o no)
- Area (rural o urbana)
- Modalidad
- Jornada

### Visión general 

In [6]:
closures_education.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 624 entries, 0 to 623
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    624 non-null    datetime64[ns]
 1   Status  624 non-null    object        
dtypes: datetime64[ns](1), object(1)
memory usage: 9.9+ KB


In [8]:
closures_education['Status'].value_counts()

Closed due to COVID-19    282
Partially open            242
Academic break             74
Fully open                 26
Name: Status, dtype: int64

### Análisis unidimensional

Haz conteos para las variables categóricas

In [ ]:
monitoring_closures_ec['Status'].unique()

In [ ]:
sns.countplot(data=monitoring_closures_ec, y='Status')
plt.show()

In [ ]:
plt.figure(figsize=[14, 8])
sns.scatterplot(data=monitoring_closures_ec, x='Date', y='Status', hue='Status')
plt.xticks(rotation=90)
plt.show()

### Análisis multidimensional

## Gobierno de Ecuador

### Visión general

### Análisis unidimensional

### Análisis multidimensional

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e7b048df-fc9b-4970-aa88-df153b270a6f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>